In [14]:
!pip install -U -r requirements.txt



     |████████████████████████████████| 6.7 MB 205 kB/s eta 0:00:01
     |████████████████████████████████| 18.0 MB 179 kB/s eta 0:00:01
Requirement already up-to-date: qiskit-ibmq-provider==0.19.2 in /home/mgabijo/.local/lib/python3.8/site-packages (from -r requirements.txt (line 3)) (0.19.2)
     |████████████████████████████████| 2.0 MB 21.3 MB/s eta 0:00:01
Requirement already up-to-date: seaborn==0.10 in /home/mgabijo/.local/lib/python3.8/site-packages (from -r requirements.txt (line 6)) (0.10.0)
Requirement already up-to-date: ipywidgets==7.5.1 in /home/mgabijo/.local/lib/python3.8/site-packages (from -r requirements.txt (line 7)) (7.5.1)
Requirement already up-to-date: pylatexenc in /home/mgabijo/.local/lib/python3.8/site-packages (from -r requirements.txt (line 8)) (2.10)
  Created wheel for qiskit: filename=qiskit-0.37.2-py3-none-any.whl size=12094 sha256=7dd0daa73ae56c2aa36bbca493ed40a731b94379fa947ba7dbac109728801c8d
  Stored in directory: /home/mgabijo/.cache/pip/wheels/5c/

In [2]:
# pylint: disable=line-too-long
from qiskit.algorithms import VQE
from qiskit_nature.algorithms import (GroundStateEigensolver,
                                      NumPyMinimumEigensolverFactory)
from qiskit_nature.drivers import Molecule, UnitsType
from qiskit_nature.drivers.second_quantization import PySCFDriver, ElectronicStructureMoleculeDriver, ElectronicStructureDriverType
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import ParityMapper
# pylint: enable=line-too-long

import matplotlib.pyplot as plt
import numpy as np
from qiskit_nature.circuit.library import UCCSD, HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.optimizers import COBYLA, SPSA, SLSQP
from qiskit.opflow import TwoQubitReduction
from qiskit import BasicAer, Aer
from qiskit.utils import QuantumInstance
from qiskit.utils.mitigation import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel

# Drug design

In [10]:

def get_qubit_op(dist):
    # Define Molecule
    molecule = Molecule(
        # Coordinates in Angstrom
        geometry=[
            ["Li", [0.0, 0.0, 0.0]],
            ["H", [dist, 0.0, 0.0]]
        ],
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )

    driver = ElectronicStructureMoleculeDriver(
        molecule=molecule,
        basis="sto3g",
        driver_type=ElectronicStructureDriverType.PYSCF)

    # Get properties
    properties = driver.run()
    num_particles = (properties
                        .get_property("ParticleNumber")
                        .num_particles)
    num_spin_orbitals = int(properties
                            .get_property("ParticleNumber")
                            .num_spin_orbitals)

    # Define Problem, Use freeze core approximation, remove orbitals.
    problem = ElectronicStructureProblem(
        driver,
        [FreezeCoreTransformer(freeze_core=True,
                               remove_orbitals=[-3,-2])])

    second_q_ops = problem.second_q_ops()  # Get 2nd Quant OP
    num_spin_orbitals = problem.num_spin_orbitals
    num_particles = problem.num_particles
    #print(second_q_ops)
    # Get Hamiltonian
    mapper = ParityMapper()  # Set Mapper
    hamiltonian = second_q_ops[0]  # Set Hamiltonian
    # Do two qubit reduction
    converter = QubitConverter(mapper,two_qubit_reduction=True)
    reducer = TwoQubitReduction(num_particles)
    qubit_op = converter.convert(hamiltonian)
    qubit_op = reducer.convert(qubit_op)

    return qubit_op, num_particles, num_spin_orbitals, problem, converter

In [11]:
def exact_solver(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

backend = BasicAer.get_backend("statevector_simulator")
distances = np.arange(0.5, 4.0, 0.2)
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=5)

# pylint: disable=undefined-loop-variable
for dist in distances:
    (qubit_op, num_particles, num_spin_orbitals,
                             problem, converter) = get_qubit_op(dist)
    result = exact_solver(problem,converter)
    exact_energies.append(result.total_energies[0].real)
    init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
    var_form = UCCSD(converter,
                     num_particles,
                     num_spin_orbitals,
                     initial_state=init_state)
    vqe = VQE(var_form, optimizer, quantum_instance=backend)
    vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
    vqe_result = problem.interpret(vqe_calc).total_energies[0].real
    vqe_energies.append(vqe_result)
    print(f"Interatomic Distance: {np.round(dist, 2)}",
          f"VQE Result: {vqe_result:.5f}",
          f"Exact Energy: {exact_energies[-1]:.5f}")

print("All energies have been calculated")

Interatomic Distance: 0.5 VQE Result: -7.04751 Exact Energy: -7.04791
Interatomic Distance: 0.7 VQE Result: -7.50392 Exact Energy: -7.50400
Interatomic Distance: 0.9 VQE Result: -7.72278 Exact Energy: -7.72283
Interatomic Distance: 1.1 VQE Result: -7.82516 Exact Energy: -7.82520
Interatomic Distance: 1.3 VQE Result: -7.86886 Exact Energy: -7.86890
Interatomic Distance: 1.5 VQE Result: -7.88210 Exact Energy: -7.88214
Interatomic Distance: 1.7 VQE Result: -7.87917 Exact Energy: -7.87920


In [2]:
# oxalic acid geometry
#       O
#       |
#       C - C - O - H
#       |   
#       O - H
# Bond lengths in Angstroms
# C-O bond length is 1.43 Angstroms 
# C-C bond length is 1.45 Angstroms
# O-H bond length is 0.96 Angstroms

# set the Molecule for Oxalic Acid
geometry = [('C', (0., 0., 0.)),
            ('O', (0., 0., 1.43)),
            ('O', (0., 0., -1.43)),
            ('C', (1.45, 0., 0.)),
            ('O', (2.48, 0., 0.)),
            ('H', (3.44, 0., 0.)),
            ('H', (0., 0.96, -1.43))]

basis = 'sto-3g'
multiplicity = 1
charge = 0

# set the Molecule for Oxalic Acid
molecule = Molecule(geometry, basis, multiplicity, charge)


In [9]:
# Define Molecule
molecule = Molecule(
    # Coordinates in Angstrom
    geometry=[
        ["Li", [0.0, 0.0, 0.0]],
        ["H", [0.7, 0.0, 0.0]]
    ],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)

driver = ElectronicStructureMoleculeDriver(
    molecule=molecule,
    basis="sto3g",
    driver_type=ElectronicStructureDriverType.PYSCF)

# Get properties
properties = driver.run()
num_particles = (properties
                    .get_property("ParticleNumber")
                    .num_particles)
num_spin_orbitals = int(properties
                        .get_property("ParticleNumber")
                        .num_spin_orbitals)

# Define Problem, Use freeze core approximation, remove orbitals.
problem = ElectronicStructureProblem(
    driver,
    [FreezeCoreTransformer(freeze_core=True,
                            remove_orbitals=[-3,-2])])

second_q_ops = problem.second_q_ops()  # Get 2nd Quant OP

In [12]:
list(second_q_ops)

['ParticleNumber',
 'ElectronicEnergy',
 'DipoleMomentX',
 'DipoleMomentY',
 'DipoleMomentZ',
 'AngularMomentum',
 'Magnetization']

In [13]:
mapper